In [1]:
# just ignore this
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'pos'))

from pos import http
from pos import config
from pos.models import db

class Config(config.Config):
        SQLALCHEMY_DATABASE_URI = 'sqlite:///:memory:'
        
app = http.create_app(Config)
app.app_context().push()
db.create_all()

# Relationship

Relationships bisa dilakukan dengan menggunakan fungsi `relationship()` dan  `sqlalchemy.schema.ForeignKey`.

Misal pada kasus ini kita akan membuat table `transaction` dan `transaction_products`, dengan schema sebagai berikut.

![Relation](img/relations.png)

### SQL

```sql

CREATE TABLE `pos`.`transactions` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `transactionscol` DATETIME NOT NULL,
PRIMARY KEY (`id`));

CREATE TABLE `pos`.`transaction_products` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `transaction_id` INT NOT NULL,
  `product_id` INT NOT NULL,
  `product_qty` INT NOT NULL,
  PRIMARY KEY (`id`));

```

### Model

```python
# pos/models/transactions.py

class Transactions(db.Model):

    id = db.Column(db.Integer, primary_key=True, autoincrement=True)

    create_on = db.Column(db.DateTime, nullable=False)

    def __init__(self):
        self.create_on = datetime.now()

```


```python
# pos/models/transactions_products.py

class TransactionProducts(db.Model):

    id = db.Column(db.Integer, primary_key=True, autoincrement=True)

    transaction_id = db.Column(db.Integer, db.ForeignKey("transactions.id"), nullable=False)

    product_id = db.Column(db.Integer, db.ForeignKey("products.id"), nullable=False)

    product_qty = db.Column(db.Integer, nullable=False)

    transaction = relationship("Transactions", backref="transaction_products")

    product = relationship("Products", backref="transaction_products")

```
Perhatikan `ForeignKey` yang merujuk ke kunci tamu, dan `relationship` yang digunakan untuk menghubungkan ke model lain.

## CRUD

Untuk operasi insert, update, delete, select sama seperti model pada biasanya, hanya saja dengan ini kita bisa langsung mengakses object lain yang berelasi.

contoh:

In [6]:
from pos.models.products import Products
from pos.models.transaction import Transactions
from pos.models.transaction_products import TransactionProducts

# membuat product
food = Products()
food.name = "Makanan"
food.price = 5000
food.stock = 10

beverage = Products()
beverage.name = "Minuman"
beverage.price = 10000
beverage.stock = 10

db.session.add_all([food, beverage])
db.session.flush()

In [7]:
# membuat transaksi
transaction = Transactions()
db.session.add(transaction)
db.session.flush()

In [8]:
# menghubungkan dengan detail transaksi
transaction_products = TransactionProducts()
transaction_products.transaction_id = transaction.id
transaction_products.product_id = food.id
transaction_products.product_qty = 2

transaction_products2 = TransactionProducts()
transaction_products2.transaction_id = transaction.id
transaction_products2.product_id = beverage.id
transaction_products2.product_qty = 2

db.session.add_all([transaction_products, transaction_products2])
db.session.commit()

In [11]:
# mengambil semua product di transaksi
# perhatikan backref pada relationship di model, dengan menggunakan backref('transaction_products') berarti 
# kita bisa membuat model transaction mengakses `TransactionProducts` dengan keyword `transaction_products`
transaction.transaction_products